In [2]:
!pip install cplex
!pip install docplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
import numpy as np
import matplotlib.pyplot as plt
import time
import docplex.mp.model as md
from scipy.spatial import Delaunay
import random
import math
from itertools import permutations

---

**k-Sum (NP-Complete Problem)**

**Goal:** From a set $U$ of $n$ numbers, find a subset $S$ of any size with sum $k$.

Place a qubit $x_{i}$ on each number $u_i$.

$$
H= \underbrace{\left( \sum_{i=1}^n u_ix_i - k \right)^2}_{\text{finds subset of sum $k$}}$$


---

In [4]:
np.random.seed(30)

In [5]:
n,k=100,np.random.randint(1,1000)
u=np.random.randint(1,100,n)
#print('k: ',k)
#print('U: ',u)
M=np.max(u)
u=u/M
k=k/M

In [6]:
model=md.Model(name='k-sum',parameters={})
model.set_time_limit(10)
x = np.array(model.binary_var_list(n))
H = (np.sum([ [u[i]*x[i]] for i in range(n) ])-k)**2
model.minimize(H)
solution = np.array(model.solve().get_value_list(x.flatten()))
print(solution)

[0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [7]:
print('U: ',u*M)
print('Size: ',np.sum(solution))
print('k: ',k*M)
print('Sum: ', np.sum(u[i]*M for i in range(n) if solution[i]==1))

U:  [38. 46. 46. 13. 24.  3. 54. 18. 47.  4. 42.  8. 66. 50. 46. 62. 36. 19.
 19. 77. 17.  7. 63. 28. 47. 46. 65. 63. 12. 16. 24. 14. 51. 34. 56. 29.
 59. 92. 79. 58. 76. 96. 45. 38. 76. 58. 40. 10. 34. 48. 40. 37. 23. 41.
 26. 55. 70. 91. 27. 79. 92. 20. 31. 50. 94. 53. 45. 62. 35. 80. 44. 29.
 61. 77. 52. 42. 57. 17. 40. 34. 10. 12. 93.  5. 99. 29. 11. 80. 18. 59.
 10. 46. 72. 49. 56. 81. 20. 46. 32. 24.]
Size:  20.0
k:  806.0
Sum:  806.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  after removing the cwd from sys.path.


In [8]:
#vars(model)

---

---

**k-Sum Group of Size p (NP-Complete Problem)**

**Goal:** From a set $U$ of $n$ numbers, find a subset $S$ of size $p$ with sum $k$.

Place a qubit $x_{i}$ on each number $u_i$.

$$
H= A \underbrace{\left( \sum_{i=1}^n u_ix_i - k \right)^2}_{\text{finds subset of sum $k$}} + B \underbrace{\left( \sum_i^n x_i -p \right)^2}_{\text{ensures subset is of size $p$}}$$


---

In [9]:
m,k_2,p,A,B=100,np.random.randint(100,1000),np.random.randint(1,20),1,100
u_2=np.random.randint(1,100,m)
#print('k: ',k_2)
#print('p: ',p)
#print('U: ',u_2)
M_2=np.max(u_2)
u_2=u_2/M_2
k_2=k_2/M_2

In [10]:
model=md.Model(name='k-sum_size_p',parameters={})
model.set_time_limit(20)
x = np.array(model.binary_var_list(m))
H = A*(np.sum([ [u_2[i]*x[i]] for i in range(m) ])-k_2)**2 + B*(np.sum([ [x[i]] for i in range(m) ])-p)**2
model.minimize(H)
solution = np.array(model.solve().get_value_list(x.flatten()))
print(np.round(solution))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0.]


In [11]:
print('U: ',u_2*M_2)
print('p: ',p)
print('Size: ',np.round(np.sum(solution)))
t=np.round(k_2*M_2)
print('k: ',t)
a=np.sum(u_2[i]*M_2 for i in range(n) if solution[i]==1)
print('Sum: ', a)
print('Difference: ',np.abs(t-a))

U:  [30. 20. 77. 52. 86. 56. 53. 24. 62.  7. 40. 99. 75. 31. 45. 84. 99. 48.
 79. 14.  8. 16. 94. 90. 91. 20. 23. 36. 71.  5. 77. 83. 23. 67. 92. 42.
 77. 94. 52. 33. 15. 57. 90. 83. 82. 24. 74. 54. 98. 95. 69. 87. 57. 92.
 39. 17. 33. 32. 67. 23. 43. 83. 73. 70. 85. 10.  3.  8. 92. 50.  4. 13.
 23.  1. 82. 18. 20. 62.  6. 62.  7.  6.  5. 80. 66. 90. 69. 76. 66. 44.
 38. 15. 16. 56. 90. 48. 11. 20. 92. 67.]
p:  13
Size:  13.0
k:  111.0
Sum:  111.0
Difference:  0.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


---

---

**Maximum Equal k-sum Subsequences (NP-Complete Problem)**

**Goal:** From a set $U$ of $n$ numbers, find $k$ subsets $S_i$ with sum $s$.

Place a qubit $x_{ij}$ on each number $u_i$ and each subset $S_j$.

$$
H= A \underbrace{\sum_{j=1}^k \left( \sum_{i=1}^n u_ix_{ij} - s \right)^2}_{\text{esnures each group has sum s}} + B \underbrace{\sum_{i=1}^n \left( \sum_{j=1}^k x_{ij} - 1 \right)^2}_{\text{ensures each element is part of a group}}$$

---

Here, the sum $s$ is the sum of all numbers in $U$ divided by the number of groups $s = \frac{\sum_i U_i}{k}$.

In [149]:
l,k_3,A,B=7,3,10,1
U=np.random.randint(1,10,l)

# Force the sum of all elements in U to be exactly divisible by k
extra=np.sum(U)%k_3
print('Mod: ',extra)

if extra==0:
  s=int(np.sum(U)/k_3)
elif extra!=0:
  # Increase sum of U by making up for the mod term exactly k-extra times
  for i in range(k_3-extra):
    U=np.append(U,1)
s=np.sum(U)/k_3
print('k: ',k_3)
print('s: ',s)
print('U: ',U)
X=np.max(U)
U=U/X
s=s/X

Mod:  0
k:  3
s:  10.0
U:  [3 2 2 8 4 8 3]


In [150]:
model=md.Model(name='s-sum_k_groups',parameters={})
model.set_time_limit(20)
x = np.array(model.binary_var_list(l*k_3)).reshape(l,k_3)
H1 = A*np.sum([ (np.sum([ [ U[i]*x[i,j]] for i in range(l)])-s)**2 for j in range(k_3)])
H2 = B*np.sum([ (np.sum(x[i,:])-1)**2 for i in range(l)])
H=H1+H2
model.minimize(H)
solution = np.array(model.solve().get_value_list(x.flatten())).reshape(l,k_3)
print(solution)

[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [151]:
sum=s*X
Universe=U*X
print('k: ',k_3)
print('s: ',sum)
print('U: ',Universe)
groups=[]
for j in range(k_3):
  groups.append([])
  for i in range(l):
    if solution[i,j]==1:
      groups[j].append(Universe[i])

for j in range(k_3):
  print('Group '+str(j)+' has sum = ',np.sum(groups[j]),' with difference from s '+str(sum-np.sum(groups[j])),'.')


k:  3
s:  10.0
U:  [3. 2. 2. 8. 4. 8. 3.]
Group 0 has sum =  10.0  with difference from s 0.0 .
Group 1 has sum =  10.0  with difference from s 0.0 .
Group 2 has sum =  10.0  with difference from s 0.0 .
